In [ ]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
import scipy.stats
from sklearn.decomposition import FastICA, PCA
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.random_projection import GaussianRandomProjection

In [ ]:
from data_helpers.wine_quality_data_helper import load_wine_quality_data
from data_helpers.mnist_data_helper import load_mnist_data

In [ ]:
class ICAWrapper:
    NAME = 'ICA'
    def __init__(self, n_components) -> None:
        self.model = FastICA(n_components=n_components, max_iter=1000)
    
    def fit(self, X):
        return self.model.fit(X)
    
    def fit_transform(self, X):
        return self.model.fit_transform(X)
    
    def transform(self, X):
        return self.model.transform(X)

class PCAWrapper:
    NAME = 'PCA'
    def __init__(self, n_components) -> None:
        self.model = PCA(n_components=n_components)

    def fit(self, X):
        return self.model.fit(X)

    def fit_transform(self, X):
        return self.model.fit_transform(X)

    def transform(self, X):
        return self.model.transform(X)

class RPWrapper:
    NAME = 'RP'
    def __init__(self, n_components) -> None:
        self.model = GaussianRandomProjection(n_components=n_components)
    
    def fit(self, X):
        return self.model.fit(X)
    
    def fit_transform(self, X):
        return self.model.fit_transform(X)

    def transform(self, X):
        return self.model.transform(X)

    def get_reconstruction_error(self, X):
        X_star = self.model.fit_transform(X)
        X_reconstructed = self.model.inverse_transform(X_star)
        return np.mean(np.square(X - X_reconstructed))
    
class LLEWrapper:
    NAME = 'LLE'
    def __init__(self, n_components) -> None:
        self.model = LocallyLinearEmbedding(n_components=n_components, n_neighbors=10)
    
    def fit(self, X):
        return self.model.fit(X)
    
    def fit_transform(self, X):
        return self.model.fit_transform(X)
    
    def transform(self, X):
        return self.model.transform(X)
    
    def get_reconstruction_error(self):
        return self.model.reconstruction_error_

In [ ]:
DATASET_NAME = 'mnist'
# DATASET_NAME = 'wine_quality'

METHOD = RPWrapper
K_FOLDS = 5
SAVE = True

In [ ]:
if DATASET_NAME == 'wine_quality':
    DATASET_STR = 'Wine Quality'
    X, y, _, _ = load_wine_quality_data()
elif DATASET_NAME == 'mnist':
    DATASET_STR = 'MNIST'
    X, y, _, _ = load_mnist_data()
else:
    raise ValueError(f'Invalid dataset name {DATASET_NAME}')

In [ ]:
df_path = f'results/{DATASET_NAME}/{METHOD.NAME}_metrics.csv'
if os.path.exists(df_path):
    df = pd.read_csv(df_path)
    df.set_index('num_components', inplace=True)
    df['num_components'] = df.index
else:
    df = pd.DataFrame()

In [ ]:
n = X.shape[0] // K_FOLDS

if DATASET_NAME == 'wine_quality':
    range_ = range(2, 12)
elif DATASET_NAME == 'mnist':
    # range_ = [2, 5, 10, 25, 50, 75, 100, 125, 150, 175, 200]
    range_ = list(range(2, 221))
else:
    raise ValueError(f'Invalid dataset name {DATASET_NAME}')

for components in tqdm(range_):
    if components in df.index:
        continue
    print(components)
    training_time_list = []
    evaluation_time_list = []
    kurtosis_list = []
    reconstruction_error_list = []
    acc_list = []
    for i in range(K_FOLDS):
        X_train = np.concatenate([X[:i * n], X[(i + 1) * n:]])
        y_train = np.concatenate([y[:i * n], y[(i + 1) * n:]])
        X_test = X[i * n:(i + 1) * n]
        y_test = y[i * n:(i + 1) * n]

        model = METHOD(components)
        t0 = time.perf_counter()
        X_train_star = model.fit_transform(X_train)
        t1 = time.perf_counter()
        training_time_list.append(t1 - t0)

        t0 = time.perf_counter()
        X_test_star = model.transform(X_test) 
        t1 = time.perf_counter()
        evaluation_time_list.append(t1 - t0)

        if DATASET_NAME == 'mnist':
            knn = KNeighborsClassifier(n_neighbors=3)
            knn.fit(model.transform(X_train), y_train)
            acc_knn = knn.score(model.transform(X_test), y_test)
            acc_list.append(acc_knn)
        elif DATASET_NAME == 'wine_quality':
            # run KNN regression
            knn = KNeighborsRegressor(n_neighbors=3)
            knn.fit(model.transform(X_train), y_train)
            knn_pred = knn.predict(model.transform(X_test))
            # get loss
            loss = np.mean(np.square(knn_pred - y_test))
            acc_list.append(loss)

        if METHOD == ICAWrapper:
            # calculate kurtosis using X_test_star
            kurtosis = scipy.stats.kurtosis(X_test_star, axis=0)
            kurtosis_list.append(kurtosis)

        if METHOD == RPWrapper:
            error = model.get_reconstruction_error(X_test)
            reconstruction_error_list.append(error)

        if METHOD == LLEWrapper:
            error = model.get_reconstruction_error()
            reconstruction_error_list.append(error)

    df.loc[components, 'num_components'] = components
    df.loc[components, 'training_time_mean'] = np.mean(training_time_list)
    df.loc[components, 'training_time_std'] = np.std(training_time_list)
    df.loc[components, 'evaluation_time_mean'] = np.mean(evaluation_time_list)
    df.loc[components, 'evaluation_time_std'] = np.std(evaluation_time_list)

    if DATASET_NAME == 'mnist':
        df.loc[components, 'accuracy_mean'] = np.mean(acc_list)
        df.loc[components, 'accuracy_std'] = np.std(acc_list)
    elif DATASET_NAME == 'wine_quality':
        df.loc[components, 'mse_mean'] = np.mean(acc_list)
        df.loc[components, 'mse_std'] = np.std(acc_list)

    if METHOD == RPWrapper or METHOD == LLEWrapper:
        df.loc[components, 'reconstruction_error_mean'] = np.mean(reconstruction_error_list)
        df.loc[components, 'reconstruction_error_std'] = np.std(reconstruction_error_list)
    if METHOD == ICAWrapper:
        df.loc[components, 'kurtosis_mean'] = np.mean(kurtosis_list)
        df.loc[components, 'kurtosis_std'] = np.std(kurtosis_list)

In [ ]:
if SAVE:
    os.makedirs(os.path.dirname(df_path), exist_ok=True)
    df.to_csv(df_path, index=False)